In [1]:
#Import required libraries
import gspread
import sys
from oauth2client.service_account import ServiceAccountCredentials
import pandas as pd
import numpy as np
import urllib
import sqlalchemy
from gspread_dataframe import set_with_dataframe
from gspread_dataframe import get_as_dataframe

In [2]:
#Import project specific functions
from column_map import column_map
from yesno_functions import *

In [3]:
#Import shared functions
sys.path.append('../..')
from IPM_Shared_Code_public.Python.google_creds_functions import create_assertion_session
from IPM_Shared_Code_public.Python.utils import get_config
from IPM_Shared_Code_public.Python.delta_functions import *
from IPM_Shared_Code_public.Python.sql_functions import *

C:\ProgramData\Anaconda3\lib\site-packages\authlib\client\__init__.py:12: AuthlibDeprecationWarning: Deprecate "authlib.client", USE "authlib.integrations.requests_client" instead.
It will be compatible before version 1.0.
Read more <https://git.io/Jeclj#file-rn-md>
  deprecate('Deprecate "authlib.client", USE "authlib.integrations.requests_client" instead.', '1.0', 'Jeclj', 'rn')


### Use the config file to setup connections

In [4]:
config = get_config('c:\Projects\config.ini')

driver = config['srv']['driver']
server = config['srv']['server']
dwh = config['db']['crowdsdb']
cred_file = config['google']['path_to_file']

In [5]:
con_string = 'Driver={' + driver + '};Server=' + server +';Database=' + dwh + ';Trusted_Connection=Yes;'
params = urllib.parse.quote_plus(con_string)
engine = sqlalchemy.create_engine("mssql+pyodbc:///?odbc_connect=%s" % params)

### Execute the function to get the renamed columns for this sheet

In [6]:
#Call the column map function to get the dictionary to be used for renaming and subsetting the columns
col_rename = column_map('crowds_dpr')

In [7]:
#Because of duplicate column names these columns are renamed based on the column index and the keys and 
#values need to be switched
col_rename = {v[0]: k for k, v in col_rename.items()}

In [8]:
cols = list(col_rename.values())

### Read the site reference list from SQL

In [9]:
sql = 'select * from crowdsdb.dbo.tbl_ref_sites'

In [10]:
site_ref = pd.read_sql(con = engine, sql = sql)[['site_id', 'desc_location']]

### Read the current data from SQL

In [11]:
sql = 'select * from crowdsdb.dbo.tbl_dpr_crowds'

In [12]:
crowds_sql = (pd.read_sql(con = engine, sql = sql)
              .drop(columns = ['crowds_id'])
              .fillna(value = np.nan, axis = 1))

In [13]:
sql_cols = list(crowds_sql.columns.values)

In [14]:
hash_rows(crowds_sql, exclude_cols = ['site_id', 'encounter_timestamp'], hash_name = 'row_hash')

### Read the latest data from Google Sheets

In [15]:
scope = ['https://spreadsheets.google.com/feeds',
         'https://www.googleapis.com/auth/drive']
creds = ServiceAccountCredentials.from_json_keyfile_name(cred_file, scope)
client = gspread.authorize(creds)

In [16]:
sheet = client.open('Crowds_Combined')

In [17]:
ws = sheet.worksheet('Sheet1')

In [24]:
crowds = (get_as_dataframe(ws, evaluate_formulas = True, header= None)
          #Always remove row 0 with the column headers
          .iloc[1:]
          .rename(columns = col_rename)
          .fillna(value = np.nan, axis = 1))[cols]

In [25]:
#Remove the rows where there timestamp is null because these sheets have extra rows full of nulls
crowds = crowds[crowds['encounter_timestamp'].notnull()]

In [26]:
crowds.head()

,encounter_timestamp,park_district,patroncount,in_playground,action_taken,amenity,comments,desc_location,borough
1,5/26/2020 17:37:04,Q-06,25.0,No,Did not approach the crowd; the crowd remains,Chess area,Chess players,Q207 | MacDonald Park,Queens
2,5/26/2020 17:17:41,X-14,17.0,No,Did not approach the crowd; the crowd remains,Soccer field,NaN,X147A-ZN01 | Starlight Park Zone 1,Bronx
3,5/26/2020 16:31:03,X-14,8.0,Yes,Did not approach the crowd; the crowd remains,Playground,On the swings and play equipment,X147A-ZN01 | Starlight Park Zone 1,Bronx
4,5/26/2020 10:55:57,X-06,20.0,No,Approached the crowd; they ignored the employee,Bench/sitting area,NaN,X010A | Tremont Park,Bronx
5,5/26/2020 10:24:04,B-13,20.0,No,Approached the crowd; they ignored the employee,Soccer field,NaN,B129 | Kaiser Park,Brooklyn


In [27]:
yesno = ['in_playground']

In [28]:
yesno_cols(crowds, yesno)

In [29]:
crowds = crowds.merge(site_ref, how = 'left', on = 'desc_location')[sql_cols]

In [26]:
hash_rows(crowds, exclude_cols = ['site_id', 'encounter_timestamp'], hash_name = 'row_hash')

In [27]:
crowds.head()

,encounter_timestamp,park_district,patroncount,in_playground,action_taken,amenity,comments,site_id,borough,row_hash
0,5/26/2020 17:37:04,Q-06,25.0,0.0,Did not approach the crowd; the crowd remains,Chess area,Chess players,Q207,Queens,230d771c770588d196f0ff3a045f014e7d2ae717c19870...
1,5/26/2020 17:17:41,X-14,17.0,0.0,Did not approach the crowd; the crowd remains,Soccer field,NaN,X147A-ZN01,Bronx,357cf0fd8cb2692275585adf6bb3a8ed856235187eb0d3...
2,5/26/2020 16:31:03,X-14,8.0,1.0,Did not approach the crowd; the crowd remains,Playground,On the swings and play equipment,X147A-ZN01,Bronx,12f45e93d05d3fb926d66c6eb1aeeab06d4e83e90a1596...
3,5/26/2020 10:55:57,X-06,20.0,0.0,Approached the crowd; they ignored the employee,Bench/sitting area,NaN,X010A,Bronx,f60be131fa74dff78e843dc7964b929c48959e6999b8ba...
4,5/26/2020 10:24:04,B-13,20.0,0.0,Approached the crowd; they ignored the employee,Soccer field,NaN,B129,Brooklyn,fa5f2cd5e2b38dcc295e39ed620afd980f2bea75f26991...


### Find the deltas based on the row hashes

In [33]:
crowds_deltas = (check_deltas(new_df = crowds, old_df = crowds_sql, on = ['encounter_timestamp', 'site_id'], 
                              hash_name = 'row_hash', dml_col = 'dml_verb'))[sql_cols + ['dml_verb']]

In [34]:
crowds_deltas.head()

,encounter_timestamp,park_district,patroncount,in_playground,action_taken,amenity,comments,site_id,borough,dml_verb
0,5/26/2020 17:37:04,Q-06,25.0,0.0,Did not approach the crowd; the crowd remains,Chess area,Chess players,Q207,Queens,I
1,5/26/2020 17:17:41,X-14,17.0,0.0,Did not approach the crowd; the crowd remains,Soccer field,NaN,X147A-ZN01,Bronx,I
2,5/26/2020 16:31:03,X-14,8.0,1.0,Did not approach the crowd; the crowd remains,Playground,On the swings and play equipment,X147A-ZN01,Bronx,I
3,5/26/2020 10:55:57,X-06,20.0,0.0,Approached the crowd; they ignored the employee,Bench/sitting area,NaN,X010A,Bronx,I
4,5/26/2020 10:24:04,B-13,20.0,0.0,Approached the crowd; they ignored the employee,Soccer field,NaN,B129,Brooklyn,I


In [39]:
crowds_inserts = crowds_deltas[crowds_deltas['dml_verb'] == 'I'][sql_cols]

In [40]:
crowds_inserts.head()

,encounter_timestamp,park_district,patroncount,in_playground,action_taken,amenity,comments,site_id,borough
0,5/26/2020 17:37:04,Q-06,25.0,0.0,Did not approach the crowd; the crowd remains,Chess area,Chess players,Q207,Queens
1,5/26/2020 17:17:41,X-14,17.0,0.0,Did not approach the crowd; the crowd remains,Soccer field,NaN,X147A-ZN01,Bronx
2,5/26/2020 16:31:03,X-14,8.0,1.0,Did not approach the crowd; the crowd remains,Playground,On the swings and play equipment,X147A-ZN01,Bronx
3,5/26/2020 10:55:57,X-06,20.0,0.0,Approached the crowd; they ignored the employee,Bench/sitting area,NaN,X010A,Bronx
4,5/26/2020 10:24:04,B-13,20.0,0.0,Approached the crowd; they ignored the employee,Soccer field,NaN,B129,Brooklyn


In [41]:
crowds_inserts.to_sql('tbl_dpr_crowds', engine, index = False, if_exists = 'append')

In [43]:
crowds_updates = crowds_deltas[crowds_deltas['dml_verb'] == 'U'][sql_cols]

In [44]:
crowds_updates.head()

,encounter_timestamp,park_district,patroncount,in_playground,action_taken,amenity,comments,site_id,borough


In [47]:
sql_update(crowds_updates, 'tbl_dpr_crowds', engine, ['encounter_timestamp', 'site_id'])